In [1]:
from mlflow import MlflowClient
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor


# Initiate remote host

In [12]:
# For Railway deployment, write the Procfile
# mlflow server --host 127.0.0.1 --port 8080

# Remote Connect

In [2]:
client = MlflowClient(tracking_uri="https://mlflowtemplate-production.up.railway.app/") # url of the mlflow server
all_experiments = client.search_experiments()

print(all_experiments)


[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1708015045472, experiment_id='1', last_update_time=1708015045472, lifecycle_stage='active', name='MLflow Quickstart', tags={}>]


# Remote test

In [3]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)


# Step 4 - Log the model and its metadata to MLflow

In [4]:
mlflow.set_tracking_uri(uri="https://mlflowtemplate-production.up.railway.app/")
# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )


2024/02/15 12:29:26 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: list index out of range
c:\Users\l501l\AppData\Local\Programs\Python\Python38\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2024/02/15 12:29:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 2
Created version '2' of model 'tracking-quickstart'.


# Load Model back

In [7]:
# 
model_info.model_uri


'runs:/68996b1241264091a4b38a5960a188c1/iris_model'

In [9]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [10]:
# Load the model back for predictions as a generic Python Function model
mlflow.set_tracking_uri(uri="https://mlflowtemplate-production.up.railway.app/")
loaded_model = mlflow.pyfunc.load_model('runs:/68996b1241264091a4b38a5960a188c1/iris_model')

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]


c:\Users\l501l\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1


## Download Model from remote

In [9]:
import mlflow

mlflow.set_tracking_uri(uri="https://mlflowtemplate-production.up.railway.app/")
mlflow_artifact_uri='runs:/4b50598b930148b89d5cbfe7c733afcd/iris_model'
local_path = mlflow.artifacts.download_artifacts(mlflow_artifact_uri)
model = mlflow.sklearn.load_model(local_path)
model

LogisticRegression(max_iter=1000, random_state=8888)

# End point (Different instance)
* Need to deploy with other instance but using the same mlflow tracking server

In [8]:
# Data
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [10]:
import requests
import json

DEPLOYED_ENDPOINT = "https://mlflowmodelhost-production.up.railway.app//invocations"
headers = {"Content-Type": "application/json"}

# Example input data for inference
input_data = {"inputs": X_test.tolist()}  # Convert input data to list for JSON serialization

# Convert input data to JSON format
input_data_json = json.dumps(input_data)

try:
    # Make a POST request to the model endpoint
    response = requests.post(url=DEPLOYED_ENDPOINT, data=input_data_json, headers=headers)

    # Print the response status code and content
    print("Response Status Code:", response.status_code)
    print("Response Content:", response.text)

    # Check the response
    if response.status_code == 200:
        # Successful request
        result = pd.DataFrame(response.json(), columns=iris_feature_names)
        result["actual_class"] = y_test
        result["predicted_class"] = result["prediction"]  # Assuming the response has a "prediction" key
        print(result[:4])
    else:
        # Handle error
        print("Error:", response.text)

except Exception as e:
    # Handle exceptions
    print("Exception:", str(e))


Response Status Code: 200
Response Content: {"predictions": [1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]}
Exception: 'prediction'
